In [1]:
import os
import sys
notebook_path = os.path.dirname(os.path.abspath("__file__"))
# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
code_path = os.path.join(core_path, 'code')

sys.path.append(core_path)
sys.path.append(code_path)

# import the wsl_terminal code
from wsl_terminal import BashConfigure
from wsl_utils import get_windows_filename, get_ubuntu_path

## Creating Function to make Running MountainSort Commands Easy

In [2]:
def run_pipeline_js(pipeline, inputs, outputs, parameters=None, verbose=False):
    """
    This is a function that will run a pipeline given the inputs, outputs, and parameters
    """
    
    command = 'ml-run-process %s ' % (pipeline)
    
    command += '--inputs '

    for key, value in inputs.items():
        if type(value) != list:
            command += '%s:%s ' % (str(key), str(value))
        else:
            for x in value:
                command += '%s:%s ' % (str(key), str(x))

    command += '--outputs '

    for key, value in outputs.items():
        command += '%s:%s ' % (str(key), str(value))

    if parameters is not None:
        command += '--parameters '

        for key, value in parameters.items():
            command += '%s:%s ' % (str(key), str(value))

    if verbose:
        print(command)

    cfg = BashConfigure()

    cfg.win32_wsl_open_bash(None, [
        command,
        'sleep 2'  # you can comment this out, I like to visualize the results for a brief moment
    ], None)

## Create Wavform Data

In [26]:
# Create some random spike waveforms

'''
This will create a random spike waveform  4 channels (M), 20 waveforms by default, with
upsamplefac*T (T=500 by default) samples, and average peak amplitude of 100.
'''

pipeline = 'ephys.synthesize_random_waveforms'  # the pipeline that creates the random waveforms

# we don't need inputs for this piepline
inputs = {}

waveforms_out = get_ubuntu_path(r'D:\dataset\waveforms_true.mda')
geometry_out = get_ubuntu_path(r'D:\dataset\geom.csv')

# determining where to save our random spike waveform files
outputs = {'waveforms_out': waveforms_out,
          'geometry_out': geometry_out}

# adding an upsampling factor of 13, 4 channels, and avgerage peak amplitude of 100
parameters = {'upsamplefac': 13,
             'M': 4,
             'average_peak_amplitude': 100}

run_pipeline_js(pipeline, inputs, outputs, parameters, verbose=True)

ml-run-process ephys.synthesize_random_waveforms --inputs --outputs waveforms_out:/mnt/d/dataset/waveforms_true.mda geometry_out:/mnt/d/dataset/geom.csv --parameters upsamplefac:13 M:4 average_peak_amplitude:100 


## Determine Spike Times

In [27]:
# Create random firing event timings

'''
This will create an output .mda with a 3xL matrix size. L is the # of events.
Second row are time stamps, third row are integer unit labels.
'''

pipeline = 'ephys.synthesize_random_firings'  # the pipeline that creates the random firings

# we don't need inputs for this pipeline
inputs = {}

firings_out = get_ubuntu_path(r'D:\dataset\firings.mda')

# determining where to save the firing events file
outputs = {'firings_out': firings_out}

# adding 600 seconds as the duration
parameters = {'duration': 600}

run_pipeline_js(pipeline, inputs, outputs, parameters, verbose=True)

ml-run-process ephys.synthesize_random_firings --inputs --outputs firings_out:/mnt/d/dataset/firings.mda --parameters duration:600 


## Create Timeseries Data for the Waveforms above at the given spike times

In [28]:
# Make a synthetic ephys dataset

pipeline = 'ephys.synthesize_timeseries'  # the pipeline that creates the ephys data

# we don't need inputs for this pipeline

inputs = {'firings': firings_out,
         'waveforms': waveforms_out}

timeseries_out = get_ubuntu_path(r'D:\dataset\timeseries.mda')

# determining where to save the timeseries file
outputs = {'timeseries_out': timeseries_out}

# adding 600 seconds as the duration
parameters = {'duration': 600,  # same duration we used for the firing rate
             'waveform_upsamplefac': 13,  # the same upsamplefac we used when creating spike waveforms
             'noise_level': 10}

run_pipeline_js(pipeline, inputs, outputs, parameters, verbose=True)

ml-run-process ephys.synthesize_timeseries --inputs firings:/mnt/d/dataset/firings.mda waveforms:/mnt/d/dataset/waveforms_true.mda --outputs timeseries_out:/mnt/d/dataset/timeseries.mda --parameters duration:600 waveform_upsamplefac:13 noise_level:10 


## Viewing Data - MountainView

In [12]:
timeseries_out = get_ubuntu_path(r'D:\dataset\timeseries.mda')
firings_out = get_ubuntu_path(r'D:\dataset\firings.mda')

view_command = 'qt-mountainview --raw %s --samplerate 30000 --firings %s' % (timeseries_out, 
                                                                             firings_out)

In [13]:
cfg = BashConfigure()
cfg.win32_wsl_open_bash(None, [view_command,
                               'sleep 2'  # you can comment this out, I like to visualize the results for a brief moment
                              ], None)

0

## Viewing Data - 

In [15]:
view_command = 'ev-timeseries %s --firings %s --samplerate=30000' % (timeseries_out, firings_out)

In [11]:
cfg = BashConfigure()
cfg.win32_wsl_open_bash(None, [view_command,
                               'sleep 2'  # you can comment this out, I like to visualize the results for a brief moment
                              ], None)

0

In [16]:
view_command

'ev-timeseries /mnt/d/dataset/timeseries.mda --firings /mnt/d/dataset/firings.mda --samplerate=30000'